# TM10007 Assignment ECG - group 3

### Data loading, save features and labels into dataframe 

In [6]:
import zipfile
import os
import pandas as pd

with zipfile.ZipFile('./ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('./ecg/')

data = pd.read_csv('./ecg/ecg_data.csv', index_col=0)

file = './ecg/ecg_data.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

y = data.label
X = data.drop('label', axis=1)

The number of samples: 827
The number of columns: 9001


### Learning Curve function 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA 
from sklearn.feature_selection import SelectKBest, chi2, SequentialFeatureSelector
from sklearn.naive_bayes import GaussianNB
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

def plot_learning_curve(clf, X_train, y_train):
    train_size, train_scores, test_scores = learning_curve(clf, X, y, verbose=2)
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(12, 8))
    plt.title(f'Learning curve for {str(type(clf[-1])).split(".")[-1][:-2]}')
    plt.plot(train_size, train_mean, color='b', label='training')
    plt.plot(train_size, test_mean, color='orange', label='validation')

    plt.fill_between(train_size, train_mean-train_std, train_mean+train_std, color='b', alpha=0.3)
    plt.fill_between(train_size, test_mean-test_std, test_mean+test_std, color='orange', alpha=0.3)
    
    plt.grid()
    plt.ylim([0, 1.1])
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.legend()
    plt.show()

# Random forest classifier 

### Loop of test-training  and evaluation loop

In [18]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA 
from sklearn.feature_selection import SelectKBest, chi2, SequentialFeatureSelector
from sklearn.naive_bayes import GaussianNB
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

rs = ShuffleSplit(n_splits=5, test_size=.3) ## moeten voordat we hem inleveren nog random state toevoegen! 

scores = {'f1': []}    ## Wat is een andere score die wat meer zegt dan accuracy? Of alleen F1? 
fitted_clf_list = []
n = -1
for train_index, test_index in rs.split(X):

    # Data split
    X_train = X.iloc[train_index]
    y_train = y[train_index]
    X_test = X.iloc[test_index]
    y_test = y[test_index]
    n+= 1 

    # PCA plot of training data
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_scaled = scaler.transform(X_train)

# we dachten PCA scatterplots van verschillende data toevoegen in subplot 
    #pca = decomposition.PCA(n_components=3)
    #pca.fit(X_scaled)
    #X_pca = pca.transform(X_scaled)
    #ax = plt.subplot(3, 2, n + 1)
    #seaborn.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y_train, ax=ax)

    # Preprocessing
    scaler = StandardScaler()
    random_forest = RandomForestClassifier()
    pca = PCA()

    # Pipeline
    pipe = Pipeline(steps=[('scaler', scaler), ("random_forest", random_forest)])

    # GridSearch
    parameters = {'random_forest__n_estimators': [50, 100, 200], 'random_forest__max_features': ['sqrt', 'log2', 0.1], 'random_forest__min_samples_leaf': [1,3,5]} 
    search = RandomizedSearchCV(pipe, parameters, cv=3, scoring = 'f1', verbose=2, n_iter= 10)
    search.fit(X_train, y_train)
    best_clf = search.best_estimator_
    fitted_clf_list.append(search)

    #plot_learning_curve(best_clf, X, y) 
    y_pred = best_clf.predict(X_test)
    scores['f1'] += [f1_score(y_test, y_pred)]
# Lukt het jullie misschien om alle learning curves (dus 5) die uit deze loop komen in een subplot te zetten? 
#       for X_train, Y_train in zip(Xs, Ys):
#      # Split data in training and testing
#        title = str(type(best_clf))
#       ax = fig.add_subplot(5, 1, num + 1)
#       plot_learning_curve(clf, title, X, Y, ax, ylim=(0.3, 1.01), cv=cv)
#       num += 1

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=3, random_forest__n_estimators=500; total time=   1.7s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=3, random_forest__n_estimators=500; total time=   1.7s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=3, random_forest__n_estimators=500; total time=   2.0s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=1, random_forest__n_estimators=100; total time=   0.6s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=1, random_forest__n_estimators=100; total time=   0.6s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=1, random_forest__n_estimators=100; total time=   0.5s
[CV] END random_forest__max_features=log2, random_forest__min_samples_leaf=5, random_forest__n_estimators=100; total time=   0.5s
[CV] END random_forest__max_f

In [20]:
scores['f1']
# variabiliteit is heeel groot tussen de verschillende test-train splits; hoe komt dat? Hij lijkt niet stabiel. 

[0.13333333333333333,
 0.2222222222222222,
 0.30434782608695654,
 0.21428571428571425,
 0.2222222222222222]

In [36]:
# Denk een goede om van de 5 loops met verschillende test-train split de cv_results overzichtelijk te laten zien; 
# dan krijgen we inzicht in welke hyperparameters betere en minder goed werken; hier begin gemaakt maar hadden geen tijd om het af te maken 
import pandas as pd
for clf in fitted_clf_list: 
    print((pd.DataFrame(clf.cv_results_))[['mean_fit_time', 'params', 'mean_test_score']])

    mean_fit_time                                             params  \
0        1.713838  {'random_forest__n_estimators': 500, 'random_f...   
1        0.514197  {'random_forest__n_estimators': 100, 'random_f...   
2        0.430410  {'random_forest__n_estimators': 100, 'random_f...   
3       11.006663  {'random_forest__n_estimators': 100, 'random_f...   
4       15.326657  {'random_forest__n_estimators': 100, 'random_f...   
5       52.916623  {'random_forest__n_estimators': 500, 'random_f...   
6        6.198839  {'random_forest__n_estimators': 50, 'random_fo...   
7       12.257258  {'random_forest__n_estimators': 100, 'random_f...   
8        7.053809  {'random_forest__n_estimators': 50, 'random_fo...   
9       61.610767  {'random_forest__n_estimators': 500, 'random_f...   
10       1.411759  {'random_forest__n_estimators': 100, 'random_f...   
11       5.432394  {'random_forest__n_estimators': 50, 'random_fo...   
12       1.793901  {'random_forest__n_estimators': 500, 'random_

# NEXT CLASSIFIER